# Uniqueness Studies 

In this notebook, we study the exact reconstruction from noiseless measurements, to understand better how many and what kind of measurements we need to reconstruct a trajectory. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

## Example Setup

A sample setup and reconstruction result.

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import DIM

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_positions = 10 #number of robot sample positions
n_complexity = 3 #model complexity

trajectory = Trajectory(n_complexity, dim=DIM, model='polynomial')
trajectory.set_coeffs(seed=2)
environment = Environment(n_anchors)

basis = trajectory.get_basis(n_samples=n_positions)

sample_points = trajectory.get_sampling_points(basis=basis)

D_complete = get_D_topright(environment.anchors, sample_points)
mask = create_mask(n_positions, n_anchors, strategy='single', 
                   dim=DIM, n_complexity=n_complexity, seed=2)
D_missing = np.multiply(D_complete, mask)

X = exactSolution(D_missing, environment.anchors, basis, 'least_squares', verbose=True)
traj = trajectory.copy()
traj.set_coeffs(coeffs=X)

environment.set_random_anchors(seed=2)
plt.figure()
environment.plot()
trajectory.plot(basis, color='orange', label='ground truth')
traj.plot(basis, color='blue', marker='x', linestyle=':', label='reconstructed')
plt.legend()

## Simulations

Looks like everything is working. Now let's try many random setups and reconstructions. 

A few first observations from cell below: 

* It looks like when we use exactly K*D measurements, such as with strategy 'simple', there is almost never a unique solution. To test, set n_positions to anything above 4, and use strategy 'simple'.
* When we have 2 * n_complexity + 1 measurements, even if at each point we only see one anchor, there is always a unique solution. To see this, use strategy 'single' and set n_positions to 2 * n_complexity + 1. If we only remove one measurement (n_positions = 2 * n_complexity) this starts breaking. 


In [ ]:
n_it = 100

# Problem: this actually returns garbage if the first estimate is not feasible. 
# See here: https://github.com/scipy/scipy/issues/7618
#method = 'minimize'  

# Problem: this returns the least squares estimate, but does not impose the quadratic constraints. 
# the residuals of this might be non-zero.
method = 'least_squares'

# Problem: produces memory error even for small problem size. 
#method = 'grid'

# Problem: can only take into account exactly K*dim constraints. 
#method = 'roots'


# the first point sees dim+1 anchors, the next K-2 points see the first two anchors, the Kth point sees only one. 
#strategy = 'simple'

# each point sees exactly one anchor. at least d+1 anchors are seen.
strategy = 'single'

# uniformly delete a few 
#strategy = 'uniform'

n_positions = 2 * n_complexity

for i in range(n_it):
    # create random trajectory
    trajectory.set_coeffs(seed=i)
    basis = trajectory.get_basis(n_samples=n_positions)
    sample_points = trajectory.get_sampling_points(basis=basis)

    D_complete = get_D_topright(environment.anchors, sample_points)
    mask = create_mask(n_positions, n_anchors, strategy=strategy, dim=DIM, n_complexity=n_complexity, seed=i)
    
    # trying to get more unique solutions by adding 
    # mask[-1, -1] = 1.0
    
    D_missing = np.multiply(D_complete, mask)

    try:
        X = exactSolution(D_missing, environment.anchors, basis, method, verbose=True)
        assert np.allclose(X, trajectory.coeffs, atol=1e-3)
        
        #X_noiseless = exactSolution(D_complete, environment.anchors, basis, method)
        #assert np.allclose(X_noiseless, trajectory.coeffs)
        print('Seed {} ok.'.format(i))
        
    except ValueError as e:
        print('ValueError for seed {}:{}'.format(i, e))
        
    except AssertionError as e:
        print('Result not exact for seed:', X, trajectory.coeffs)
        
        # We found an example of inexact reconstruction. 
        # We can thus stop here and continue plotting and 
        # investigating it. 
        break
        
    except Exception as e:
        print('Singular matrix for seed {}? Error message:'.format(i)) 
        raise e

if method == 'roots':
    from exact_solution import objective_root
    print('objective_root found:')
    print(objective_root(X, environment.anchors, basis, D_missing))
    print('objective_root original:')
    print(objective_root(trajectory.coeffs, environment.anchors, basis, D_missing))

In [ ]:
traj_second_solution = trajectory.copy()
traj_second_solution.set_coeffs(coeffs=X)

plt.figure()
plt.title('plot of two ambiguous solutions')

trajectory.plot(basis, mask=mask, color='green')
trajectory.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
trajectory.plot_number_measurements(basis, mask, legend=True)

traj_second_solution.plot(basis, mask=mask, color='red')
traj_second_solution.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
traj_second_solution.plot_number_measurements(basis, mask, legend=False)
environment.plot()
environment.annotate()

plt.axis('equal')
#plt.xlim([0, 6])
#plt.ylim([0, 6])